# UTILITIES

## GET DUMMIES 

Just get the dummies

In [ ]:
titanic_features = ['sex', 'age', 'sibsp', 'parch', 'fare']
X = pd.get_dummies(titanic[titanic_features])

Keep the previous data

In [ ]:
categorical_features = [
 'animal_type',
 'intake_condition',
 'intake_type',
 'sex_upon_intake',
 'sex_upon_outcome',
 'outcome_type'
]
data_dummies = pd.get_dummies(original_data, columns=categorical_features, drop_first=True)

## STANDARIZE

In [ ]:
to_std_columns = ["intake_year", "intake_number", "age_upon_intake_(years)", "time_in_shelter_days", "age_upon_outcome_(years)"]

data_dummies[to_std_columns] = (data_dummies[to_std_columns] - data_dummies[to_std_columns].mean(axis=0)) / data_dummies[to_std_columns].std(axis=0)

## TRAIN TEST SPLIT

In [ ]:
def train_test_spli(data, X_remove, y_col, proportion: float = 0.8, seed: int = 42):
    random.seed(seed)
    N = len(data)
    indices = list(range(N))
    random.shuffle(indices)
    split = int(N*proportion)

    X = data.drop(columns=X_remove)
    y = data[y_col]
    X_train, y_train = X.iloc[:split], y.iloc[:split]
    X_test , y_test  = X.iloc[split:], y.iloc[split:]
    
    print(f"Training with {len(X_train)} samples")
    print(f"Testing with {len(X_test)} samples")
    return X_train, y_train, X_test , y_test

X_train, y_train, X_test , y_test = train_test_spli(data_dummies, X_remove=['adopted', 'outcome_type'], y_col="adopted")

## CONFUSION MATRIX + FSCORE

In [ ]:
def confusion_matrix(y_pred, y_true, verbose: bool = True, plot: bool = False):
    tp = np.sum((y_pred == 1) & (y_true == 1))  # True Positives
    tn = np.sum((y_pred == 0) & (y_true == 0))  # True Negatives
    fp = np.sum((y_pred == 1) & (y_true == 0))  # False Positives
    fn = np.sum((y_pred == 0) & (y_true == 1))  # False Negatives

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    
    if verbose:
        print(f"{tp = :6d} | {fp = :6d}")
        print(f"----------- |  -----------")
        print(f"{fn = :6d} | {tn = :6d}")
        print(f"{precision = :.4}")
        print(f"{recall = :.4}")
        print(f"{f_score = :.4}")
        
    if plot:
        confusion_matrix = np.array([[tp, fn],  
                                    [fp, tn]]) 

        confusion_matrix = pd.DataFrame(confusion_matrix, 
                                    index=["Actual Positive", "Actual Negative"], 
                                    columns=["Predicted Positive", "Predicted Negative"])

        plt.figure(figsize=(8, 6))
        sn.heatmap(confusion_matrix, annot=True, fmt="d", cmap='YlGnBu', cbar=False)
        plt.title("Confusion Matrix")
        plt.ylabel("Actual")
        plt.xlabel("Predicted")
        plt.tight_layout()
        plt.show()


Try different thresholds

In [ ]:
f_scores = []
precisions = []
recalls = []
thresholds = [i*0.1 for i in range(11)]
# thresholds = np.linspace(0, 1, 100)
for threshold in thresholds:
    pred = distribution[:,1] > threshold
    f_score, precision, recall = confusion_matrix(pred, y_test, verbose=False, plot=False)
    f_scores += [f_score]
    precisions += [precision]
    recalls += [recall]
    
plt.figure(figsize=(8,6))

plt.plot(thresholds, f_scores, marker="o", label="f_scores")
plt.plot(thresholds, precisions, marker="o", label="precisions")
plt.plot(thresholds, recalls, marker="o", label="recalls")
# plt.plot(thresholds, f_scores, marker="", label="f_scores")
# plt.plot(thresholds, precisions, marker="", label="precisions")
# plt.plot(thresholds, recalls, marker="", label="recalls")

plt.xlabel('Thresholds')
plt.ylabel('Score')
plt.title(f'Score vs Thresholds')
# plt.ylim(0.9, 1.0)
plt.xlim(thresholds[0] - 0.1, thresholds[-1] + .1)
plt.tight_layout()
plt.legend()


# Regression 1

you do not need a train/test split if you want to get the propensy score. Train and apply the model on the entire dataset. If you're wondering why this is the right thing to do in this situation, recall that the propensity score model is not used in order to make predictions about unseen data. Its sole purpose is to balance the dataset across treatment groups.
(See p. 74 of Rosenbaum's book for an explanation why slight overfitting is even good for propensity scores.

### `STANDARIZE`

In [ ]:
def standarize_column(names: list[str], df: pd.DataFrame):
    for name in names:
        df[name] = (df[name] - df[name].mean())/df[name].std()
        
standarize_column(["age", "creatinine_phosphokinase", "ejection_fraction", "platelets", "serum_creatinine", "serum_sodium"], df)

### Part 1 Linear regression: Modelling time spent at the hospital

- We will perform a regression analysis to model the number of days spent at the hospital, among the population of patients.


- To get started with our model, we need two components:

   1. The equation describing the model
   2. The data
   
   
- Equations are specified using patsy formula syntax. Important operators are:
    1. `~` : Separates the left-hand side and right-hand side of a formula.
    2. `+` : Creates a union of terms that are included in the model.
    3. `:` : Interaction term.
    3. `*` : `a * b` is short-hand for `a + b + a:b`, and is useful for the common case of wanting to include all interactions between a set of variables.
    
    
- Intercepts are added by default.


- Categorical variables can be included directly by adding a term C(a). More on that soon!


- For (2), we can conveniently use pandas dataframe.

### An example

- Let's start with an example from our dataset. We are interested in two predictors: diabetes and high blood pressure. These are the two predictors that we want to use to fit the outcome, the number of days spent at the hospital, using a linear regression.

- A model that achieves this is formulated as:
        time ~ C(diabetes) + C(high_blood_pressure)
        
- We can create this model using smf.ols().

- OLS stands for ordinary least squares linear regression.

- The two components: the formula and the data are stated explicitly.

- The terms in the formula are columns in pandas dataframe. Easy!

### ordinal least Squares 

In [ ]:
mod = smf.ols(formula='time ~ C(diabetes) + C(high_blood_pressure)', data=df)
mod = smf.ols(formula='time ~ C(high_blood_pressure) * C(DEATH_EVENT,  Treatment(reference=0)) + C(diabetes)', data=df)

res = mod.fit()
print(res.summary())
# Confident intervals
res.conf_int()

**Comment:** In the first model, `high_blood_pressure` is associated with an additive coefficient of around -25. Thus, in the model, whenever a patient has high blood pressure we deduce -25 days out of the prediction.
In the second model, `high_blood_pressure` is associated with an multiplicative coefficient of around -0.22. This means that, in the model, whenever a patient has high blood pressure we multiply his or her outcome by $e^{-0.22} \simeq 0.80$. 

### Logistic regression

This coeficients show how the `log-odds` will change when unitary change of the variable happens
- To go from p to odds: $\text{odds} = \frac{p}{1 - p}$
- To go from p to log-odds: $\text{log-odds} = \ln\left(\frac{p}{1 - p}\right)$
- To go from log-odds to odds: $\text{odds} = e^{\text{log-odds}}$
- To go from log-odds to p: $p = \frac{1}{1 + e^{-\text{log-odds}}}$
- To go from odds to log-odds: $\text{log-odds} = \ln(\text{odds})$
- To go from odds to p: $p = \frac{\text{odds}}{1 + \text{odds}}$

In [ ]:
def p_to_log_odd(p):
    return  np.log(p/(1-p))
def log_odd_to_p(log_odd):
    return 1/(1+np.exp(-log_odd))

p = 0.1
original_log_odd = p_to_log_odd(p)
new_log_odd = original_log_odd + 0.66

new_p = log_odd_to_p(new_log_odd)
print(f"{p = }")
print(f"{original_log_odd = }")
print(f"{new_log_odd = }")
print(f"{new_p = }, increased by {p-new_p} \n")

In [ ]:
mod = smf.logit(formula='DEATH_EVENT ~  age + creatinine_phosphokinase + ejection_fraction + \
                        platelets + serum_creatinine + serum_sodium + \
                        C(diabetes) + C(high_blood_pressure) +\
                        C(sex) + C(anaemia) + C(smoking) + C(high_blood_pressure)', data=df)
res = mod.fit()
print(res.summary())

##### A lot of useful information is provided by default.

`Coef` is the value assigned to each `variable` + the `bias`

- The dependent variable : time (number of days at the hospital)
- Method: The type of model that was fitted (OLS)
- Nb observations: The number of datapoints (299 patients)
- R2: The fraction of explained variance
- A list of predictors
- For each predictor: coefficient, standard error of the coefficients, p-value, 95% confidence intervals. 
  - We can see that one is a significant predictor if p < 0.5 (or very small numbers)
- Warnings if there are numerical issues (hopefully not!)

### Predict / Propensity Score

In [ ]:
df["propensity_score"] = res.predict()

### Get the values

In [ ]:
variables = res.params.index
coefficients = res.params.values

for coefficient, name in zip(coefficients, variables):
    if name != "high_blood_pressure": continue
    print(f"A change in {name} of 1 unit, increases your dead probability by ~{coefficient: 0.4f}")


In [ ]:
# feature names
variables = res.params.index

# quantifying uncertainty!

# coefficients
coefficients = res.params.values

# p-values
p_values = res.pvalues

# standard errors
standard_errors = res.bse.values

l1, l2, l3, l4 = zip(*sorted(zip(coefficients[1:], variables[1:], standard_errors[1:], p_values[1:])))
# fancy plotting 
plt.errorbar(l1, np.array(range(len(l1))), xerr= 2*np.array(l3), linewidth = 1,
             linestyle = 'none',marker = 'o',markersize= 3,
             markerfacecolor = 'black',markeredgecolor = 'black', capsize= 5)

plt.vlines(0,0, len(l1), linestyle = '--')

plt.yticks(range(len(l2)),l2)

## Matching propensity

In [ ]:
df["propensity_score"] = res.predict()
treated = df[df["treat"] == 1]
control = df[df["treat"] == 0]

def similarity(p_1, p_2):
    return 1-np.abs(p_1 - p_2)

# THE HIGHER THE EASIER FOR THEM TO MATCH
def get_similarity(control_row, treatment_row):
    return(
        1
        - np.abs(control_row["propensity_score"] - treatment_row["propensity_score"])
        - int(control_row["black"]  !=  treatment_row["black"])
        - int(control_row["hispan"]  !=  treatment_row["hispan"])
    )

In [ ]:
# Create an empty undirected graph
G = nx.Graph()

# Loop through all the pairs of instances
for control_id, control_row in control_df.iterrows():
    for treatment_id, treatment_row in treatment_df.iterrows():

        # Calculate the similarity 
        similarity = similarity(
            # control_row['Propensity_score'], treatment_row['Propensity_score']
            control_row, treatment_row
        )

        # Add an edge between the two instances weighted by the similarity between them
        G.add_weighted_edges_from([(control_id, treatment_id, similarity)])

# Generate and return the maximum weight matching on the generated graph
matching = nx.max_weight_matching(G)

matched = [i[0] for i in list(matching)] + [i[1] for i in list(matching)]

balanced_df_1 = lalonde_data.iloc[matched]

# Regression 2

### Logistic regression

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve

In [ ]:
logistic = LogisticRegression(solver='lbfgs')
logistic.fit(X.values, y)

### Evaluation

men = np.array[[...],[...]]
pred = logistic.predict([men])
distribution = logistic.predict_proba([men])
error = mean_squared_error(y, pred)

### Cross Validation

In [ ]:
y_pred = cross_val_predict(logistic, X, y, cv=10, method="predict_proba")
precision = cross_val_score(logistic, X, y, cv=10, scoring="precision").mean()
recall = cross_val_score(logistic, X, y, cv=10, scoring="recall").mean()

### ROC Curve

In [ ]:
# Predict the probabilities with a cross validationn
y_pred = cross_val_predict(logistic, X, y, cv=10, method="predict_proba")
# Compute the False Positive Rate and True Positive Rate
fpr, tpr, _ = roc_curve(y, y_pred[:, 1])
# Compute the area under the fpt-tpf curve
auc_score = auc(fpr, tpr)

# K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(3) #k
model.fit(X, y_moons)
Z = model.predict([X[0]])
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
